In [1]:
import hydra
from hydra import compose, initialize
from conf.register_configs import register_configs
from utils.factories import InferenceBootstrapper,SimWorkerFactory
from utils.inference_core import run_inference_driver, create_shard_iterator
import ray
# Register our custom variants and resolvers
register_configs()

/root/conda/envs/vlm_node_1016/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-11 07:39:43,612	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Initialize Hydra manually (replaces @hydra.main)
# 'config_path' is relative to this notebook
with initialize(version_base=None, config_path="conf"):
    # Here you can list overrides just like you would on the CLI
    cfg = compose(config_name="inference_config", overrides=[
        "task.run_name=test",
        "rollout.max_steps=100",
        "vlm.save_outputs=True", # need this for RL
        
    ])

print(f"Model ID: {cfg.vlm.model_id}")

Model ID: Phyllis1/qwen3_sft_sft_sparse_03drop_single_action_20260103_210803_ckpt10800


In [3]:
bootstrapper = InferenceBootstrapper(cfg)
bootstrapper.setup_cluster()

2026-01-11 07:39:51,722	INFO worker.py:1998 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/root/conda/envs/vlm_node_1016/lib/python3.10/site-packages/ray/_private/worker.py:2046: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


In [4]:
sim_ref = SimWorkerFactory.create(bootstrapper.resolved_dict['sim'],bootstrapper.typed_cfg.resources,bootstrapper.typed_cfg.task,None)[0]

In [5]:
ray.get(sim_ref.assign_shard.remote(None))

(HabitatRayWorker pid=2736680) PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
(HabitatRayWorker pid=2736680) PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
(HabitatRayWorker pid=2736680) PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
(HabitatRayWorker pid=2736680) PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
(HabitatRayWorker pid=2736680) PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
(HabitatRayWorker pid=2736680) PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring
(HabitatRayWorker pid=2736680) 2026-01-11 07:39:59,296 Initializing dataset ObjectNav-v1


(HabitatRayWorker pid=2736680) skipping sim initialization since no shards provided, please call assign_shard


In [6]:
from utils.inference_core import InferenceWorker,RLWorker
from utils.vlm_worker import VLMTrainingConfig
from peft import LoraConfig
training_config = VLMTrainingConfig(peft_config=LoraConfig(
                r=128, lora_alpha=256, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM",
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                # modules_to_save=["multi_modal_projector"],
                modules_to_save=["spatial_head"], 
    ))
trainer = RLWorker(bootstrapper.resolved_dict['rollout'],**bootstrapper.resolved_dict['vlm'])
trainer.setup_training(
    config=training_config,
    rank=0,
    world_size=1,
    master_addr="localhost",
    master_port=29500
)

(HabitatRayWorker pid=2736680) Actor assigned with shard of 2000 episodes.
Loading Phyllis1/qwen3_sft_sft_sparse_03drop_single_action_20260103_210803_ckpt10800 with sparsifying patch...


TypeError: PreTrainedModel.gradient_checkpointing_enable() got an unexpected keyword argument 'use_reentrant'

In [ ]:
trainer.run_episode(sim_ref,ray.get(sim_ref.reset.remote()))

(HabitatRayWorker pid=2691174) 2026-01-11 07:30:44,474 Video created: /Projects/spatial_training/dump/results/test/worker_0/6s7QHgap2fW/6s7QHgap2fW_61/video.mp4
(HabitatRayWorker pid=2691174) IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2626, 480) to (2640, 480) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
(HabitatRayWorker pid=2691174) [rawvideo @ 0x434891c0] Stream #0: not enough frames to estimate rate; consider increasing probesize


using normal mode
keeping 300/300
keeping 212/300
keeping 238/300
keeping 237/300
keeping 226/300
keeping 263/300
keeping 30/300
keeping 30/300
keeping 30/300
keeping 30/300
keeping 223/300
keeping 253/300
keeping 268/300
keeping 30/300
keeping 205/300
keeping 207/300
keeping 222/300
keeping 234/300
keeping 210/300
keeping 209/300
keeping 215/300
keeping 227/300
keeping 224/300
keeping 169/300
keeping 207/300
keeping 225/300
keeping 249/300
keeping 244/300
keeping 232/300
keeping 202/300
keeping 238/300
keeping 208/300
keeping 208/300
keeping 212/300
keeping 214/300
keeping 227/300
keeping 241/300
keeping 30/300
keeping 30/300
keeping 183/300
keeping 219/300
keeping 186/300
keeping 30/300
keeping 204/300
keeping 186/300
keeping 176/300
keeping 234/300
keeping 230/300
keeping 217/300
keeping 219/300
keeping 228/300
keeping 217/300
keeping 194/300
keeping 241/300
keeping 241/300
keeping 234/300
keeping 268/300
keeping 241/300
keeping 212/300
keeping 222/300
keeping 193/300
keeping 212/30

(Actor(HabitatRayWorker, ae59e85d56bc6556cc6a02a701000000),
 False,
 {'episode_label': 'ziup5kvtCCR_43',
  'spl': 0.42452780798895884,
  'success': 1.0,
  'steps': 80,
  'goal_name': 'sofa'})

In [ ]:
trainer.outputs